In [90]:
# Load your usual SpaCy model (one of SpaCy English models)
import spacy
import operator
nlp = spacy.load('en')

In [91]:
# load NeuralCoref and add it to the pipe of SpaCy's model
import neuralcoref
coref = neuralcoref.NeuralCoref(nlp.vocab, greedyness=0.8)
nlp.add_pipe(coref, name='neuralcoref')

In [92]:
q_1 = "Where was Barack Obama born?"
a = "Hawaii birth place is BARACK OBAMA"
q_2 = "Tell me more about him"

In [93]:
doc = nlp(a+q_2)

In [94]:
doc._.coref_scores

{Hawaii: {Hawaii: 1.6314061880111694},
 Hawaii birth place: {Hawaii birth place: 1.5916625261306763,
  Hawaii: -1.5772948265075684},
 BARACK OBAMATell me more about him: {BARACK OBAMATell me more about him: 2.0123772621154785,
  Hawaii: -1.5654293298721313,
  Hawaii birth place: -1.5055248737335205},
 him: {him: -0.067355215549469,
  Hawaii: -2.385866165161133,
  Hawaii birth place: -1.8358627557754517,
  BARACK OBAMATell me more about him: -1.6307239532470703}}

In [95]:
labels[0] = 

SyntaxError: invalid syntax (<ipython-input-95-2f9b70f6b74a>, line 1)

In [96]:
labels

['', '']

In [106]:
doc = nlp(q_1+q_2)

In [109]:
for cluster in list(doc._.coref_scores.keys()):
    if str(cluster) in q_2:
        print(max(doc._.coref_scores[cluster].values()))

5.035101890563965


In [107]:
doc._.coref_scores

{the wife of Barack Obama: {the wife of Barack Obama: 1.8370593786239624},
 Barack Obama: {Barack Obama: 1.2383612394332886,
  the wife of Barack Obama: -1.5839101076126099},
 her: {her: -0.4763253629207611,
  the wife of Barack Obama: 5.035101890563965,
  Barack Obama: -0.7100987434387207}}

In [110]:
doc._.coref_clusters

[Barack Obama: [the wife of Barack Obama, Barack Obama, her]]

In [111]:
# You're done. You can now use NeuralCoref the same way you usually manipulate a SpaCy document and it's annotations.
doc = nlp(a+q_2)

doc._.has_coref

True

In [112]:
for cluster in list(doc._.coref_scores.keys()):
    if str(cluster) in q_2:
        print(max(doc._.coref_scores[cluster].values()))

7.124642372131348


In [93]:
doc._.coref_scores

{It: {It: 2.066462516784668},
 Michelle Obama: {Michelle Obama: 1.499599814414978, It: -1.6225340366363525},
 her: {her: -0.4763253629207611,
  It: -2.1658437252044678,
  Michelle Obama: 6.911686897277832}}

In [94]:
doc._.coref_clusters

[Michelle Obama: [It, Michelle Obama, her]]

In [28]:
from nltk.corpus import stopwords
stopwords = stopwords.words('english')

In [31]:
stopwords+=['word']

In [32]:
stopwords

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [116]:
def execute_resolver(text):
    #TODO: question mark issue
    doc = nlp(text)
    return doc

def get_coreferent_label(question_1, question_2, answer):
    scores = [-9999, -9999]
    labels = ['', '']
    
    question_1+='. '
    answer+='. '
    
    doc_1 = nlp(question_1 + question_2)
    doc_2 = nlp(answer + question_2)
    
    print(doc_1)
    print(doc_2)
    
    if doc_1._.has_coref:
        for cluster in list(doc_1._.coref_scores.keys()):
            if str(cluster) in question_2:
                max_score = max(doc_1._.coref_scores[cluster].values())
                if max_score > scores[0]:
                    scores[0] = max_score
                    print(doc_1._.coref_scores[cluster])
                    labels[0] = str(max(doc_1._.coref_scores[cluster], key=doc_1._.coref_scores[cluster].get))

    if doc_2._.has_coref:
        for cluster in list(doc_2._.coref_scores.keys()):
            if str(cluster) in question_2:
                max_score = max(doc_2._.coref_scores[cluster].values())
                if max_score > scores[1]:
                    scores[1] = max_score
                    labels[1] = str(max(doc_2._.coref_scores[cluster], key=doc_2._.coref_scores[cluster].get))

    max_index = scores.index(max(scores))

    if scores[max_index] > -9999:
        return {'score': scores[max_index], 'label': labels[max_index], 'index': max_index}
    else:
        return None

In [117]:
get_coreferent_label(q_1, q_2, a)

Where was Barack Obama born?. Tell me more about him
Hawaii birth place is BARACK OBAMA. Tell me more about him
{him: -0.28984731435775757, Barack Obama: 1.1879937648773193}


{'score': 2.711412191390991, 'label': 'BARACK OBAMA', 'index': 1}

In [118]:
from fuzzywuzzy import fuzz

In [119]:
def match(strings_list, mask):
    ratios = list()
    for str in strings_list:
        ratios.append(fuzz.ratio(str, mask))

    return ratios.index(max(ratios)), max(ratios)

In [122]:
match(['Barack Obama'], 'Barack Obama')

(0, 100)